In [1]:
import pickle
from pyrosetta import *
from scipy.stats import norm
import MDAnalysis as mda
import numpy as np
import chiLife as xl
from rosetta.core.scoring.methods import ContextDependentTwoBodyEnergy
from itertools import chain
from time import perf_counter

init()
pose = pose_from_pdb('1omp.clean.pdb')

@EnergyMethod()
class RosettaDipolarDistance(ContextDependentTwoBodyEnergy):
    def __init__(self):
        ContextDependentTwoBodyEnergy.__init__(self, self.creator())

        # Load in experimental data
        with open('R7_Holo_Data.pkl', 'rb') as f:
            self.site_pairs = pickle.load(f)


        self.sites = tuple(set(chain.from_iterable((key for key in self.site_pairs))))
        self.SLCache = {}
        self.iteration = 0


    def setup_for_scoring(self, pose, sf):
        # Create MDA Universe if it does not exist
        if not hasattr(self, 'mda_protein'):
            self.mda_protein = xl.pose2mda(pose)


        if self.iteration % 100 == 0 and pose.is_fullatom():
            print('100 iterations passed - recalculating rotamer libraries')
            # Update the universe with rosetta coordinates
            self.mda_protein.atoms.positions = np.array([res.xyz(atom)
                                                         for res in pose.residues
                                                         for atom in range(1, res.natoms() + 1)
                                                         if res.atom_type(atom).element().strip() != 'X'])

            # Recalculate spin lables
            self.SLCache = {site: xl.SpinLabel('R7M', site, self.mda_protein) for site in self.sites}


        else:
            if self.iteration % 100 == 0:
                print('100 iterations passed in CG')
            # Just move the spin labels
            for site in self.sites:
                backbone =  np.array([pose.residue(site).xyz('N'), pose.residue(site).xyz('CA'), pose.residue(site).xyz('C')])
                self.SLCache[site]._to_site(backbone)

        self.iteration += 1



    def residue_pair_energy(self, res1, res2, pose, sf, emap):
        r1 = res1.seqpos()
        r2 = res2.seqpos()

        # Skip if it is not a restrainted residue
        if (r1, r2) not in self.site_pairs:
            score = 0.

        # Otherwise calculate the overlap between the predicted P(r) and the proivided restraint
        else:
            # Calculate preidcted P(r)s
            r, P = self.site_pairs[(r1, r2)]
            SL1, SL2 = self.SLCache[r1], self.SLCache[r2]
            dd = xl.get_dd(SL1, SL2, r)

            # Overlap Score
            score = -np.sum(np.minimum(P, dd))
            emap.set(self.scoreType, score)

    def defines_intrares_energy(self, weights):
        return False

    def atomic_interaction_cutoff(self):
        return 500.0

RDD = RosettaDipolarDistance.scoreType
sf = get_score_function()
sf.set_weight(RDD, 1.0)

relax = pyrosetta.rosetta.protocols.relax.FastRelax()
relax.set_scorefxn(sf)
relax.apply(pose)


ModuleNotFoundError: No module named 'pyrosetta'

In [26]:
import pickle
import json
import re
import numpy as np
with open('DeerData.pkl', 'rb') as f:
    Data = pickle.load(f)



In [33]:


R7HoloData = {}
for key in Data:
    sub = Data[key]
    
    site1, site2 = re.findall(r'\d+', key)
    
    for key2  in sub:
        label, state = key2
        Exp = sub[key2]
        
        if state != 'Holo' or label != 'R7':
            continue
        
        R7HoloData[(int(site1), int(site2))] = Exp.r * 10 , Exp.P / np.trapz(Exp.P, Exp.r * 10)        

In [34]:
with open('R7_Holo_Data.pkl', 'wb') as f:
    pickle.dump(R7HoloData, f)